In [ ]:
import pandas as pd
import swat

# Server setup
(requires user-specific parameters)

In [ ]:
s = swat.CAS(host, port, username)

In [ ]:
s.table.addCaslib(caslib = "data",
    dataSource = {"srcType":"path"},
    path = "/path/to/lubm_50/csv/data")

s.sessionProp.setSessOpt(cmplib="data.myRoutines")

In [ ]:
s.loadActionSet('Network')
s.loadActionSet('fcmpact')

In [ ]:
NTHREADS = 8

# Load data

In [ ]:
s.table.loadTable(caslib="data", path="lubm_50_links.csv", casout={"name":"links", "replace":True})
s.table.loadTable(caslib="data", path="lubm_50_nodes.csv", casout={"name":"nodes", "replace":True})

# Run queries

## q02

In [ ]:
nodesQuery = [[0, 'GraduateStudent'],
              [1, 'Department'],
              [2, 'University']]
linksQuery = [[0, 1, ''],
              [0, 2, 'undergraduateDegreeFrom'],
              [1, 2, 'subOrganizationOf']]
nodesQuery = pd.DataFrame(nodesQuery, columns = ['node','type'])
linksQuery = pd.DataFrame(linksQuery, columns = ['from', 'to', 'type'])

In [ ]:
s.addRoutines(
    routineCode = '''
    function myLinkFilter(fromQ, toQ, type $);
        if(fromQ=0 and toQ=1) then return (type='worksFor' or type='memberOf');
        else return (1);
        endsub;''',
    package   = "myPackage",
    saveTable = True,
    funcTable = {"name": "myRoutines", "replace": True}
)  

In [ ]:
s.upload(nodesQuery, casout = {'name':'nodesQuery', 'replace':True})
s.upload(linksQuery, casout = {'name':'linksQuery', 'replace':True})
result = s.network.patternMatch(
                nThreads       = NTHREADS,
                multiLinks     = True,
                direction      = "directed",
                links          = {"name":"links"},
                nodes          = {"name":"nodes"},
                linksQuery     = {"name":"linksQuery"},
                nodesQuery     = {"name":"nodesQuery"},
                nodesVar       = {"vars":("type")},
                linksVar       = {"vars":("type")},
                nodesQueryVar  = {"vars":("type")},
                linksQueryVar  = {"vars":("type")},
                linkFilter     = "myLinkFilter")

## q04

In [ ]:
nodesQuery = [[0, '', ''],
              [1, 'Department0.University0', 'Department']]
linksQuery = [[0, 1, 'worksFor']]
nodesQuery = pd.DataFrame(nodesQuery, columns = ['node', 'id','type'])
linksQuery = pd.DataFrame(linksQuery, columns = ['from', 'to', 'type'])

In [ ]:
s.addRoutines(
    routineCode = '''
    function myNodeFilter(nodeQ, type $);
        if(nodeQ=0) then return (type='FullProfessor' or type='AssociateProfessor' or type='AssistantProfessor');
        else return (1);
    endsub;''',
    package   = "myPackage",
    saveTable = True,
    funcTable = {"name": "myRoutines", "replace": True}
)  

In [ ]:
s.upload(nodesQuery, casout = {'name':'nodesQuery', 'replace':True})
s.upload(linksQuery, casout = {'name':'linksQuery', 'replace':True})
result = s.network.patternMatch(
                nThreads       = NTHREADS,
                multiLinks     = True,
                direction      = "directed",
                links          = {"name":"links"},
                nodes          = {"name":"nodes"},
                linksQuery     = {"name":"linksQuery"},
                nodesQuery     = {"name":"nodesQuery"},
                nodesVar       = {"vars":("id", "type")},
                linksVar       = {"vars":("type")},
                nodesQueryVar  = {"vars":("id", "type")},
                linksQueryVar  = {"vars":("type")},
                nodeFilter     = "myNodeFilter")

## q05

In [ ]:
nodesQuery = [[0, '', ''],
              [1, 'Department0.University0', 'Department']]
linksQuery = [[0, 1]]
nodesQuery = pd.DataFrame(nodesQuery, columns = ['node', 'id','type'])
linksQuery = pd.DataFrame(linksQuery, columns = ['from', 'to'])

In [ ]:
s.addRoutines(
    routineCode = '''
    function myNodeFilter(nodeQ, type $);
        if(nodeQ=0) then return (type in ('FullProfessor','AssociateProfessor','AssistantProfessor','Lecturer','GraduateStudent','UndergraduateStudent'));
        else return (1);
    endsub;
    function myLinkFilter(fromQ, toQ, type $);
        if(fromQ=0 and toQ=1) then return (type='worksFor' or type='memberOf');
        else return (1);
    endsub;''',
    package   = "myPackage",
    saveTable = True,
    funcTable = {"name": "myRoutines", "replace": True}
)

In [ ]:
s.upload(nodesQuery, casout = {'name':'nodesQuery', 'replace':True})
s.upload(linksQuery, casout = {'name':'linksQuery', 'replace':True})
result = s.network.patternMatch(
                nThreads       = NTHREADS,
                multiLinks     = True,
                direction      = "directed",
                links          = {"name":"links"},
                nodes          = {"name":"nodes"},
                linksQuery     = {"name":"linksQuery"},
                nodesQuery     = {"name":"nodesQuery"},
                nodesVar       = {"vars":("id", "type")},
                linksVar       = {"vars":("type")},
                nodesQueryVar  = {"vars":("id", "type")},
                nodeFilter     = "myNodeFilter",
                linkFilter     = "myLinkFilter")

## q06

In [ ]:
nodesQuery = [[0, '']]
nodesQuery = pd.DataFrame(nodesQuery, columns = ['node', 'type'])

In [ ]:
s.addRoutines(
    routineCode = '''
    function myNodeFilter(nodeQ, type $);
        if(nodeQ=0) then return (type='GraduateStudent' or type='UndergraduateStudent');
        else return (1);
    endsub;''',
    package   = "myPackage",
    saveTable = True,
    funcTable = {"name": "myRoutines", "replace": True}
)

In [ ]:
s.upload(nodesQuery, casout = {'name':'nodesQuery', 'replace':True})
result = s.network.patternMatch(
                nThreads       = NTHREADS,
                multiLinks     = True,
                direction      = "directed",
                links          = {"name":"links"},
                nodes          = {"name":"nodes"},
                nodesQuery     = {"name":"nodesQuery"},
                nodesVar       = {"vars":"type"},
                nodeFilter     = "myNodeFilter")

# q07

In [ ]:
nodesQuery = [[0, '', ''], 
              [1, '', 'Course'],
              [2, 'Department0.University0.AssociateProfessor0', '']]
linksQuery = [[0, 1, 'takesCourse'],
              [2, 1, 'teacherOf']]
nodesQuery = pd.DataFrame(nodesQuery, columns = ['node', 'id','type'])
linksQuery = pd.DataFrame(linksQuery, columns = ['from', 'to', 'type'])

In [ ]:
s.addRoutines(
    routineCode = '''
    function myNodeFilter(nodeQ, type $);
        if(nodeQ=0) then return (type='GraduateStudent' or type='UndergraduateStudent');
        else return (1);
    endsub;''',
    package   = "myPackage",
    saveTable = True,
    funcTable = {"name": "myRoutines", "replace": True}
)

In [ ]:
s.upload(nodesQuery, casout = {'name':'nodesQuery', 'replace':True})
s.upload(linksQuery, casout = {'name':'linksQuery', 'replace':True})
result = s.network.patternMatch(
                nThreads       = NTHREADS,
                multiLinks     = True,
                direction      = "directed",
                links          = {"name":"links"},
                nodes          = {"name":"nodes"},
                linksQuery     = {"name":"linksQuery"},
                nodesQuery     = {"name":"nodesQuery"},
                nodesVar       = {"vars":("id", "type")},
                linksVar       = {"vars":("type")},
                nodesQueryVar  = {"vars":("id", "type")},
                nodeFilter     = "myNodeFilter")

## q08

### q08-1

In [ ]:
nodesQuery = [[0, '', ''], 
              [1, '', 'Department'], 
              [2, 'University0', '']]
linksQuery = [[0, 1, ''],
              [1, 2, 'subOrganizationOf']]
nodesQuery = pd.DataFrame(nodesQuery, columns = ['node', 'id','type'])
linksQuery = pd.DataFrame(linksQuery, columns = ['from', 'to', 'type'])

In [ ]:
s.addRoutines(
    routineCode = '''
    function myNodeFilter(nodeQ, type $);
        if(nodeQ=0) then return (type='GraduateStudent' or type='UndergraduateStudent');
        else return (1);
    endsub;
    function myLinkFilter(fromQ, toQ, type $);
        if(fromQ=0 and toQ=1) then return (type='memberOf' or type='worksFor');
        else return (1);
    endsub;''',
    package   = "myPackage",
    saveTable = True,
    funcTable = {"name": "myRoutines", "replace": True}
)

In [ ]:
s.upload(nodesQuery, casout = {'name':'nodesQuery', 'replace':True})
s.upload(linksQuery, casout = {'name':'linksQuery', 'replace':True})
result = s.network.patternMatch(
                nThreads       = NTHREADS,
                multiLinks     = True,
                direction      = "directed",
                links          = {"name":"links"},
                nodes          = {"name":"nodes"},
                linksQuery     = {"name":"linksQuery"},
                nodesQuery     = {"name":"nodesQuery"},
                nodesVar       = {"vars":("id", "type")},
                linksVar       = {"vars":("type")},
                nodesQueryVar  = {"vars":("id", "type")},
                nodeFilter     = "myNodeFilter",
                linkFilter     = "myLinkFilter")

### q08-2

In [ ]:
nodesQuery = [[0, '', ''], 
              [1, '', 'Department'],
              [2, '', ''],
              [3, 'University0', '']]
linksQuery = [[0, 1, ''], 
              [1, 2, 'subOrganizationOf'],
              [2, 3, 'subOrganizationOf']]
nodesQuery = pd.DataFrame(nodesQuery, columns = ['node', 'id','type'])
linksQuery = pd.DataFrame(linksQuery, columns = ['from', 'to', 'type'])

In [ ]:
s.addRoutines(
    routineCode = '''
    function myNodeFilter(nodeQ, type $);
        if(nodeQ=0) then return (type='GraduateStudent' or type='UndergraduateStudent');
        else return (1);
    endsub;
    function myLinkFilter(fromQ, toQ, type $);
        if(fromQ=0 and toQ=1) then return (type='memberOf' or type='worksFor');
        else return (1);
    endsub;''',
    package   = "myPackage",
    saveTable = True,
    funcTable = {"name": "myRoutines", "replace": True}
)

In [ ]:
s.upload(nodesQuery, casout = {'name':'nodesQuery', 'replace':True})
s.upload(linksQuery, casout = {'name':'linksQuery', 'replace':True})
result = s.network.patternMatch(
                nThreads       = NTHREADS,
                multiLinks     = True,
                direction      = "directed",
                links          = {"name":"links"},
                nodes          = {"name":"nodes"},
                linksQuery     = {"name":"linksQuery"},
                nodesQuery     = {"name":"nodesQuery"},
                nodesVar       = {"vars":("id", "type")},
                linksVar       = {"vars":("type")},
                nodesQueryVar  = {"vars":("id", "type")},
                nodeFilter     = "myNodeFilter",
                linkFilter     = "myLinkFilter")

## q09

In [ ]:
nodesQuery = [[0], 
              [1],
              [2]]
linksQuery = [[0, 1, 'advisor'],
              [1, 2, 'teacherOf'],
              [0, 2, 'takesCourse']]
nodesQuery = pd.DataFrame(nodesQuery, columns = ['node'])
linksQuery = pd.DataFrame(linksQuery, columns = ['from', 'to', 'type'])

In [ ]:
s.addRoutines(
    routineCode = '''
    function myNodeFilter(nodeQ, type $);
        if(nodeQ=0) then return (type='GraduateStudent' or type='UndergraduateStudent');
        if(nodeQ=1) then return (type='FullProfessor' or type='AssociateProfessor' or type='AssistantProfessor' or type='Lecturer');
        if(nodeQ=3) then return (type='Course' or type='GraduateCourse');
        else return (1);
    endsub;''',
    package   = "myPackage",
    saveTable = True,
    funcTable = {"name": "myRoutines", "replace": True}
)

In [ ]:
s.upload(nodesQuery, casout = {'name':'nodesQuery', 'replace':True})
s.upload(linksQuery, casout = {'name':'linksQuery', 'replace':True})
result = s.network.patternMatch(
                nThreads       = NTHREADS,
                multiLinks     = True,
                direction      = "directed",
                links          = {"name":"links"},
                nodes          = {"name":"nodes"},
                linksQuery     = {"name":"linksQuery"},
                nodesQuery     = {"name":"nodesQuery"},
                nodesVar       = {"vars":("id", "type")},
                linksVar       = {"vars":("type")},
                linksQueryVar  = {"vars":("type")},
                nodeFilter     = "myNodeFilter")

## q12

In [ ]:
def q12(nodesQuery, linksQuery):
    s.upload(nodesQuery, casout = {'name':'nodesQuery', 'replace':True})
    s.upload(linksQuery, casout = {'name':'linksQuery', 'replace':True})
    result = s.network.patternMatch(
                nThreads       = NTHREADS,
                multiLinks     = True,
                direction      = "directed",
                links          = {"name":"links"},
                nodes          = {"name":"nodes"},
                linksQuery     = {"name":"linksQuery"},
                nodesQuery     = {"name":"nodesQuery"},
                nodesVar       = {"vars":("id", "type")},
                linksVar       = {"vars":("type")},
                nodesQueryVar  = {"vars":("id", "type")},
                linksQueryVar  = {"vars":("type")})

### q12-1

In [ ]:
nodesQuery = [[0, '', 'FullProfessor'],
              [1, '', 'Department'],
              [2, 'University0', ''],
              [3, '', 'Department']]
linksQuery = [[0, 1, 'subOrganizationOf'],
              [1, 2, 'teacherOf'],
              [0, 2, 'headOf']]
nodesQuery = pd.DataFrame(nodesQuery, columns = ['node', 'id', 'type'])
linksQuery = pd.DataFrame(linksQuery, columns = ['from', 'to', 'type'])

In [ ]:
q12(nodesQuery, linksQuery)

### q12-2

In [ ]:
nodesQuery = [[0, '', 'FullProfessor'],
              [1, '', 'Department'],
              [2, 'University0', ''],
              [3, '', 'Department'],
              [4, '','']]
linksQuery = [[0, 1, 'worksFor'],
              [1, 4, 'subOrganizationOf'],
              [0, 3, 'headOf'],
              [4, 2, 'subOrganizationOf']]
nodesQuery = pd.DataFrame(nodesQuery, columns = ['node', 'id', 'type'])
linksQuery = pd.DataFrame(linksQuery, columns = ['from', 'to', 'type'])

In [ ]:
q12(nodesQuery, linksQuery)

### q12-3

In [ ]:
nodesQuery = [[0, '', 'FullProfessor'],
              [1, '', 'Department'],
              [2, 'University0', '']]
linksQuery = [[0, 1, 'worksFor'],
              [1, 2, 'subOrganizationOf'],
              [0, 1, 'headOf']]
nodesQuery = pd.DataFrame(nodesQuery, columns = ['node', 'id', 'type'])
linksQuery = pd.DataFrame(linksQuery, columns = ['from', 'to', 'type'])

In [ ]:
q12(nodesQuery, linksQuery)

### q12-4

In [ ]:
nodesQuery = [[0, '', 'FullProfessor'],
              [1, '', 'Department'],
              [2, 'University0', ''],
              [3, '', '']]
linksQuery = [[0, 1, 'worksFor'],
              [1, 3, 'subOrganizationOf'],
              [0, 1, 'headOf'],
              [3, 2, 'subOrganizationOf']]
nodesQuery = pd.DataFrame(nodesQuery, columns = ['node', 'id', 'type'])
linksQuery = pd.DataFrame(linksQuery, columns = ['from', 'to', 'type'])

In [ ]:
q12(nodesQuery, linksQuery)

## q13

In [ ]:
nodesQuery = [[0, ''],
              [1, 'University0']]
linksQuery = [[0, 1]]
nodesQuery = pd.DataFrame(nodesQuery, columns = ['node', 'id'])
linksQuery = pd.DataFrame(linksQuery, columns = ['from', 'to'])

In [ ]:
s.addRoutines(
    routineCode = '''
    function myLinkFilter(fromQ, toQ, type $);
        if(fromQ=0 and toQ=1) then return (type='doctoralDegreeFrom' or type='mastersDegreeFrom' or type='undergraduateDegreeFrom');
        else return (1);
    endsub;''',
    package   = "myPackage",
    saveTable = True,
    funcTable = {"name": "myRoutines", "replace": True}
)

In [ ]:
s.upload(nodesQuery, casout = {'name':'nodesQuery', 'replace':True})
s.upload(linksQuery, casout = {'name':'linksQuery', 'replace':True})
result = s.network.patternMatch(
                nThreads       = NTHREADS,
                multiLinks     = True,
                direction      = "directed",
                links          = {"name":"links"},
                nodes          = {"name":"nodes"},
                linksQuery     = {"name":"linksQuery"},
                nodesQuery     = {"name":"nodesQuery"},
                nodesVar       = {"vars":("id")},
                linksVar       = {"vars":("type")},
                nodesQueryVar  = {"vars":("id")},
                linkFilter     = "myLinkFilter")

## q14

In [ ]:
nodesQuery = [[0, 'UndergraduateStudent']]
nodesQuery = pd.DataFrame(nodesQuery, columns = ['node', 'type'])

In [ ]:
s.upload(nodesQuery, casout = {'name':'nodesQuery', 'replace':True})
result = s.network.patternMatch(
                nThreads       = NTHREADS,
                multiLinks     = True,
                direction      = "directed",
                links          = {"name":"links"},
                nodes          = {"name":"nodes"},
                nodesQuery     = {"name":"nodesQuery"},
                nodesVar       = {"vars":("type")},
                nodesQueryVar  = {"vars":("type")})